In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import io
import statsmodels.formula.api as smf
import statsmodels.api as sm 
import scipy
import scipy.stats as stats
import re
import matplotlib.pyplot as plt
from statsmodels.iolib.summary2 import summary_params # вывод результатов тестирования
from statsmodels.iolib.summary2 import summary_col # вывод результатов тестирования
from statsmodels.stats.outliers_influence import variance_inflation_factor # VIF

# 2 LM/BG-тест

## 2.1 Consumption equation

In [11]:
df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/Consumption.csv')
df

,Unnamed: 0,yd,ce
0,1,59505.0,57168
1,2,59717.4,55464
2,3,59039.1,56332
3,4,61342.6,55836
4,5,60544.6,54488
...,...,...,...
195,196,390576.6,359372
196,197,391381.5,363896
197,198,389502.6,364428
198,199,388282.1,366224


In [12]:
df.diff()

,Unnamed: 0,yd,ce
0,NaN,NaN,NaN
1,1.0,212.4,-1704.0
2,1.0,-678.3,868.0
3,1.0,2303.5,-496.0
4,1.0,-798.0,-1348.0
...,...,...,...
195,1.0,2336.8,24.0
196,1.0,804.9,4524.0
197,1.0,-1878.9,532.0
198,1.0,-1220.5,1796.0


In [14]:
df['log_yd']=np.log(df['yd'])
df['log_ce']=np.log(df['ce'])
df.diff().dropna()

,Unnamed: 0,yd,ce,log_yd,log_ce
1,1.0,212.4,-1704.0,0.003563,-0.030260
2,1.0,-678.3,868.0,-0.011423,0.015529
3,1.0,2303.5,-496.0,0.038275,-0.008844
4,1.0,-798.0,-1348.0,-0.013094,-0.024438
5,1.0,-220.0,-812.0,-0.003640,-0.015015
...,...,...,...,...,...
195,1.0,2336.8,24.0,0.006001,0.000067
196,1.0,804.9,4524.0,0.002059,0.012510
197,1.0,-1878.9,532.0,-0.004812,0.001461
198,1.0,-1220.5,1796.0,-0.003138,0.004916


In [17]:
model = smf.ols(data=df.diff().dropna(), formula='log_yd~log_ce').fit()
model.params.round(2)

Intercept    0.01
log_ce       0.37
dtype: float64

In [19]:
res0 = model.resid
res0

1      0.008827
2     -0.023126
3      0.035603
4     -0.009987
5     -0.004025
         ...   
195    0.000028
196   -0.008526
197   -0.011302
198   -0.010909
199   -0.009642
Length: 199, dtype: float64

In [20]:
res1 = res0.drop(labels=[1], axis=0)

In [21]:
res2 = res1.drop(labels=[2], axis=0)
res3 = res2.drop(labels=[3], axis=0)

In [22]:
res3.reset_index(drop=True)

0     -0.009987
1     -0.004025
2      0.019736
3     -0.020565
4     -0.000112
         ...   
191    0.000028
192   -0.008526
193   -0.011302
194   -0.010909
195   -0.009642
Length: 196, dtype: float64

In [26]:
DF = df.diff().dropna()
DF['res0'] = res0.reset_index(drop=True)
DF['res1'] = res1.reset_index(drop=True)
DF['res2'] = res2.reset_index(drop=True)
DF['res3'] = res3.reset_index(drop=True)
DF.dropna(inplace=True)
DF

,Unnamed: 0,yd,ce,log_yd,log_ce,res0,res1,res2,res3
1,1.0,212.4,-1704.0,0.003563,-0.030260,-0.023126,0.035603,-0.009987,-0.004025
2,1.0,-678.3,868.0,-0.011423,0.015529,0.035603,-0.009987,-0.004025,0.019736
3,1.0,2303.5,-496.0,0.038275,-0.008844,-0.009987,-0.004025,0.019736,-0.020565
4,1.0,-798.0,-1348.0,-0.013094,-0.024438,-0.004025,0.019736,-0.020565,-0.000112
5,1.0,-220.0,-812.0,-0.003640,-0.015015,0.019736,-0.020565,-0.000112,-0.026773
...,...,...,...,...,...,...,...,...,...
191,1.0,2128.3,3196.0,0.005497,0.009014,0.003194,-0.009434,-0.014963,0.000028
192,1.0,3283.7,-692.0,0.008422,-0.001945,-0.009434,-0.014963,0.000028,-0.008526
193,1.0,-697.9,1636.0,-0.001784,0.004592,-0.014963,0.000028,-0.008526,-0.011302
194,1.0,-2610.5,2236.0,-0.006701,0.006242,0.000028,-0.008526,-0.011302,-0.010909


In [28]:
mdl = smf.ols(data=DF, formula='res0~res1+res2+res3+log_ce').fit()
mdl.rsquared

0.10802079408577336

In [29]:
R2 = mdl.rsquared
R2*(len(DF)-3)

20.739992464468486

In [30]:
Hi2 = stats.chi2.ppf(1-0.01, 3)
Hi2

11.344866730144373